In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
import torch.nn as nn
import wandb
import torch
random_seed = 42

In [94]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)

In [95]:
X_train.shape

(21551, 20)

In [96]:
train_dataset = ut.MyTensorDataset(torch.FloatTensor(X_train.values), torch.FloatTensor(Y_train.values))
test_dataset = ut.MyTensorDataset(torch.FloatTensor(X_test.values), torch.FloatTensor(Y_test.values))
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True,)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False,)

In [97]:
size = len(X_train.columns)
net = nn.Sequential(nn.Linear(size, size), nn.Linear(size, 1), nn.Sigmoid())
n_epoch = 20
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [98]:
loss_history = []

for i in range(n_epoch):
    for x_batch, y_batch in train_dataloader:
        #print(x_batch)
        #y_batch = torch.transpose(y_batch, 0, 1)[0]
        #y_batch = y_batch[0]
        #print(y_batch)
        # Forward
        predictions = net(x_batch)
        #print(predictions)
        loss = criterion(predictions, y_batch)

        # Backward
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

        loss_history.append(loss.item())

KeyboardInterrupt: 

In [ ]:
test_preds = np.array([])

for x_batch, y_batch in test_dataloader:
    predictions = net(x_batch)
    predictions = torch.transpose(predictions, 0, 1)[0]
    predictions = predictions.detach().numpy()
    test_preds = np.append(test_preds, predictions)

In [ ]:
test_preds 

array([1., 1., 1., ..., 1., 1., 1.])

In [ ]:
from sklearn.metrics import roc_auc_score
#print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, test_preds)}")

ROC-AUC score : 0.5


In [ ]:
ut.writeNN(net)